In [1]:
pip install bs4


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install gym

In [ ]:
!pip install yfinance
!pip install mplfinance

In [ ]:
pip install keras-tuner

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder with the new parameter name
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the 'Ticker' column
ticker_encoded = encoder.fit_transform(cleaned_data[['Ticker']])

# Create a DataFrame with encoded columns
ticker_encoded_df = pd.DataFrame(ticker_encoded, columns=encoder.get_feature_names_out())

# Concatenate this DataFrame with the original cleaned_data
cleaned_data = pd.concat([cleaned_data.drop('Ticker', axis=1), ticker_encoded_df], axis=1)


In [ ]:
pip install flask


In [ ]:
pip install sqlalchemy

In [ ]:
pip install cvxpy

In [ ]:
pip install blpapi


In [ ]:
pip install quantstats

In [ ]:
pip install ipympl

In [ ]:
pip install ffill

In [ ]:
pip install openpyxl

In [ ]:
pip install xlrd

In [ ]:
pip install os

In [ ]:
pip install prophet

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip uninstall sqlalchemy 2.0.31

In [ ]:
pip uninstall SQLAlchemy - y

In [ ]:
pip install --upgrade typing-extensions

In [ ]:
pip check


In [ ]:
pip install typing-extensions==4.5.0  # or any version <4.6.0 and >=3.6.6


In [ ]:
pip install KerasRegressor

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the selected tickers from different regions and sectors
selected_tickers = ['AAPL US EQUITY', 'HSI INDEX', 'NKY INDEX ', 'GOOGL', 'VOD LN EQUITY']

# Create a subplot for each selected ticker
fig = make_subplots(rows=len(selected_tickers), cols=1, shared_xaxes=True,
                    vertical_spacing=0.02, subplot_titles=[f'{ticker} Rolling Averages' for ticker in selected_tickers])

# Loop through each ticker to add traces
for i, ticker in enumerate(selected_tickers, start=1):
    # Fetch the data for the ticker
    ticker_data = cleaned_data[cleaned_data['Ticker'] == ticker].copy()
    
    if ticker_data.empty:
        print(f"No data available for {ticker}. Skipping...")
        continue

    # Calculate 50-day and 200-day moving averages
    ticker_data['50-Day MA'] = ticker_data['Last Price'].rolling(window=50).mean()
    ticker_data['200-Day MA'] = ticker_data['Last Price'].rolling(window=200).mean()

    # Add traces for Last Price, 50-day MA, and 200-day MA
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=ticker_data['Last Price'], mode='lines', name=f'{ticker} Last Price', line=dict(color='grey', width=1)), row=i, col=1)
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=ticker_data['50-Day MA'], mode='lines', name=f'{ticker} 50-Day MA', line=dict(color='blue', width=2)), row=i, col=1)
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=ticker_data['200-Day MA'], mode='lines', name=f'{ticker} 200-Day MA', line=dict(color='red', width=2)), row=i, col=1)

    # Set grid lines and update layout for each subplot
    fig.update_yaxes(showgrid=True, gridcolor='lightgray', row=i, col=1)

# Update overall layout
fig.update_layout(
    height=1800,  # Adjusted height for clarity
    width=1200,
    title_text="Trend Analysis with 50-Day and 200-Day Moving Averages for Selected Tickers",
    showlegend=True,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Show the plot
fig.show()


In [ ]:
import pandas as pd

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Ensure the DataFrame is sorted by Dates for each Ticker
cleaned_data = cleaned_data.sort_values(by=['Ticker', 'Dates'])

# Apply RSI calculation
cleaned_data['RSI'] = cleaned_data.groupby('Ticker')['Last Price'].apply(calculate_rsi).reset_index(level=0, drop=True)

# Display the first few rows of the updated DataFrame
print(cleaned_data[['Dates', 'Last Price', 'Volume', 'Ticker', 'RSI']].head(20))


In [ ]:
import pandas as pd

# Define the MACD calculation function
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Last Price'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Last Price'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    macd_hist = macd - signal
    return macd, signal, macd_hist

# Initialize lists to store the MACD values
macd_list = []
signal_list = []
hist_list = []

# Calculate MACD for each ticker
for ticker in cleaned_data['Ticker'].unique():
    ticker_data = cleaned_data[cleaned_data['Ticker'] == ticker]
    macd, signal, hist = calculate_macd(ticker_data)
    macd_list.extend(macd)
    signal_list.extend(signal)
    hist_list.extend(hist)

# Add the MACD values to the cleaned_data DataFrame
cleaned_data['MACD'] = macd_list
cleaned_data['MACD_Signal'] = signal_list
cleaned_data['MACD_Hist'] = hist_list

# Display the first few rows of the updated DataFrame
print(cleaned_data[['Dates', 'Last Price', 'Volume', 'Ticker', 'MACD', 'MACD_Signal', 'MACD_Hist']].head(20))


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the selected tickers for plotting
selected_tickers = ['AAPL US EQUITY', 'HSI INDEX', 'NKY INDEX ', 'GOOGL', 'VOD LN EQUITY']

# Define colors for the tickers
colors = ['blue', 'green', 'purple', 'orange', 'cyan']

# Create a subplot for each ticker with two rows: one for RSI and one for MACD
fig = make_subplots(rows=len(selected_tickers) * 2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.05, subplot_titles=[f'{ticker} RSI' for ticker in selected_tickers] + 
                                                      [f'{ticker} MACD' for ticker in selected_tickers])

for i, ticker in enumerate(selected_tickers):
    ticker_data = cleaned_data[cleaned_data['Ticker'] == ticker]
    
    # Plot RSI
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=ticker_data['RSI'], mode='lines', name=f'{ticker} RSI',
                             line=dict(color=colors[i])), row=i*2+1, col=1)
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=[70]*len(ticker_data), mode='lines', name='Overbought (70)',
                             line=dict(color='red', dash='dash')), row=i*2+1, col=1)
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=[30]*len(ticker_data), mode='lines', name='Oversold (30)',
                             line=dict(color='black', dash='dash')), row=i*2+1, col=1)

    # Plot MACD
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=ticker_data['MACD'], mode='lines', name=f'{ticker} MACD',
                             line=dict(color=colors[i])), row=i*2+2, col=1)
    fig.add_trace(go.Scatter(x=ticker_data['Dates'], y=ticker_data['MACD_Signal'], mode='lines', name=f'{ticker} MACD Signal',
                             line=dict(color='red')), row=i*2+2, col=1)
    fig.add_trace(go.Bar(x=ticker_data['Dates'], y=ticker_data['MACD_Hist'], name=f'{ticker} MACD Histogram',
                         marker=dict(color='green')), row=i*2+2, col=1)

# Update layout
for axis in fig.layout:
    if 'xaxis' in axis:
        fig.update_layout({axis: dict(showgrid=True, gridcolor='black')})
    if 'yaxis' in axis:
        fig.update_layout({axis: dict(showgrid=True, gridcolor='black')})

fig.update_layout(
    title='RSI and MACD for Selected Tickers',
    height=1800, width=1300,
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True,
    legend=dict(x=1, y=1)
)

# Show the plot
fig.show()


In [ ]:
# Example for LSTM model preparation
time_steps = 60  # Define the sequence length

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:(i + time_steps)].values)
        y.append(data.iloc[i + time_steps])
    return np.array(X), np.array(y)

X_train_seq, y_train_seq = create_sequences(X_train, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test, time_steps)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming 'cleaned_data' is your DataFrame and contains features like 'Last Price' and 'Volume'
features = ['Last Price', 'Volume']
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
cleaned_data[features] = scaler.fit_transform(cleaned_data[features])

# Display the scaled features
print(cleaned_data.head())


In [ ]:
# Define a split point for the last part of the data to be used as the test set
split_ratio = 0.8
split_index = int(len(cleaned_data) * split_ratio)

# Split the data
train_data = cleaned_data.iloc[:split_index]
test_data = cleaned_data.iloc[split_index:]

# Print the shapes to verify the split
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")


In [ ]:
pip install keras-tuner

In [ ]:
env = PortfolioEnv(price_data)
vec_env = DummyVecEnv([lambda: env])

# Check the environment
check_env(env)

# Train the SAC agent
model = SAC('MlpPolicy', vec_env, verbose=1)
model.learn(total_timesteps=10000)

# Save the model
model.save("sac_portfolio_management")

# Test the trained model
obs = vec_env.reset()
balances = []

for i in range(len(price_data)):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, infos = vec_env.step(action)
    current_balances = vec_env.get_attr('current_balance')
    balances.append(current_balances[0])
    if any(dones):
        break

In [ ]:
pip install stable-baselines3


In [ ]:
pip install -c intel mkl

In [ ]:
conda install numpy

In [ ]:
conda install -c intel mkl

In [ ]:
conda install -c anaconda keras

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
conda config --add channels conda-forge


In [ ]:
conda install mkl


In [ ]:
conda config --set channel_priority strict

In [ ]:
conda install numpy pandas matplotlib gym stable-baselines3


In [ ]:
conda update conda

In [ ]:
pip install update conda

In [ ]:
pip install tensorflow

In [ ]:
conda deactivate


In [ ]:
pip uninstall tensorflow keras tensorflow-macos


In [ ]:
y

In [ ]:
test_tf_keras.py

In [ ]:
# Minimal script to check TensorFlow and Keras imports
try:
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.optimizers import Adam
    print("TensorFlow and Keras imports successful")
except Exception as e:
    print(f"Error during imports: {e}")


In [ ]:
python test_tf_keras.py


In [ ]:
conda activate portfolio-optimisation-env


In [ ]:
conda activate portfolio-optimisation-env


In [ ]:
pip install ipmypl

In [ ]:
pip activate portfolio-optimisation-env

In [ ]:
import sys
print(sys.executable)


In [ ]:
pip list


In [1]:
pip install openpyxl

  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 7.3 MB/s eta 0:00:00
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gymnasium

  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
Using cached cloudpickle-3.0.0-py3-none-any.whl (20 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install stable_baselines3

  Using cached stable_baselines3-2.3.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached torch-2.3.1-cp38-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached stable_baselines3-2.3.2-py3-none-any.whl (182 kB)
Using cached torch-2.3.1-cp38-none-macosx_11_0_arm64.whl (61.0 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
Using cached sympy-1.12.1-py3-none-any.whl (5.7 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB

In [4]:
pip install logging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/vj/3n5q5bxn0vg4zl7ytlnmzjc40000gn/T/pip-install-7pmqlj00/logging_e15498298f284a00b6efe17fd6affb16/setup.py", line 3, in <module>
          setup(name = "logging",
        File "/Users/CHARLES/.pyenv/versions/3.8.12/lib/python3.8/distutils/core.py", line 134, in setup
          ok = dist.parse_command_line()
        File "/Users/CHARLES/.pyenv/versions/3.8.12/lib/python3.8/distutils/dist.py", line 483, in parse_command_line
          args = self._parse_command_opts(parser, args)
        File "/Users/CHARLES/.pyenv/versions/portfolio-optimisation-env

In [5]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install linear regression

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement regression (from versions: none)
ERROR: No matching distribution found for regression
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install svm

  Preparing metadata (setup.py) ... done
  Created wheel for svm: filename=svm-0.1.0-py3-none-any.whl size=3483 sha256=371490a7e45071e13a20247d079a9e8fd18e137e7b0ab80e60a1501199c966bc
  Stored in directory: /Users/CHARLES/Library/Caches/pip/wheels/21/49/94/366fb6396e6577b43ce54bfd19c79383594c32b6956b72960c
Successfully built svm
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install keras_tuner

  Using cached keras_tuner-1.4.7-py3-none-any.whl.metadata (5.4 kB)
  Using cached kt_legacy-1.0.5-py3-none-any.whl.metadata (221 bytes)
Using cached keras_tuner-1.4.7-py3-none-any.whl (129 kB)
Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scipy.stats

ERROR: Could not find a version that satisfies the requirement scipy.stats (from versions: none)
ERROR: No matching distribution found for scipy.stats
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install cvxopt

  Using cached cvxopt-1.3.2.tar.gz (4.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for cvxopt (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-14.5-arm64-cpython-38
      creating build/lib.macosx-14.5-arm64-cpython-38/cvxopt
      copying src/python/misc.py -> build/lib.macosx-14.5-arm64-cpython-38/cvxopt
      copying src/python/_version.py -> build/lib.macosx-14.5-arm64-cpython-38/cvxopt
      copying src/python/msk.py -> build/lib.macosx-14.5-arm64-cpython-38/cvxopt
      copying src/python/__init__.py -> build/lib.macosx-14.5-arm64-cpython-38/cvxopt
      copying src/python/solvers.py -> build/lib.macosx-14.5-arm64-cpython-38/cvxopt
      copying src/python/cvxprog.py ->

In [8]:
conda install logging

ValueError: The python kernel does not appear to be a conda environment.  Please use ``%pip install`` instead.

In [5]:
pip install dill

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install stable_baselines3

  Using cached stable_baselines3-2.3.2-py3-none-any.whl.metadata (5.1 kB)
Using cached stable_baselines3-2.3.2-py3-none-any.whl (182 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install gymnasium

  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
Using cached cloudpickle-3.0.0-py3-none-any.whl (20 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install Adam

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install  dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install scikeras

  Using cached scikeras-0.12.0-py3-none-any.whl.metadata (4.0 kB)
INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.
  Using cached scikeras-0.11.0-py3-none-any.whl.metadata (4.0 kB)
Using cached scikeras-0.11.0-py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipympl

  Using cached ipympl-0.9.3-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl.metadata (755 bytes)
  Using cached ipywidgets-8.1.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.11-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.11-py3-none-any.whl.metadata (4.1 kB)
Using cached ipympl-0.9.3-py2.py3-none-any.whl (511 kB)
Using cached ipywidgets-8.1.3-py3-none-any.whl (139 kB)
Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
Using cached jupyterlab_widgets-3.0.11-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.11-py3-none-any.whl (2.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install  Pandoc 

  Preparing metadata (setup.py) ... done
  Using cached plumbum-1.8.3-py3-none-any.whl.metadata (10 kB)
Using cached plumbum-1.8.3-py3-none-any.whl (127 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
  Created wheel for Pandoc: filename=pandoc-2.3-py3-none-any.whl size=33274 sha256=beb67119a58bd8f8d302264ad188bad42e4bb0e5c4b57a776c8a39297cd83e08
  Stored in directory: /Users/CHARLES/Library/Caches/pip/wheels/90/3a/a8/3237a93e3a6261bd24edabf3277ca59f64c1710b3d8c7c72a0
Successfully built Pandoc
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install nbconvert 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install PyQtWebEngine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.2/123.2 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 37.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 40.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 39.5 MB/s eta 0:00:0000:0100:01
  Created wheel for PyQt5-sip: filename=PyQt5_sip-12.13.0-cp38-cp38-macosx_14_0_arm64.whl size=79647 sha256=0ce4909501fb5c79e422a8cd0aabddf4221ad46dec8aa5bd2aefdb24d1157464
  Stored in directory: /Users/CHARLES/Library/Caches/pip/wheels/7f/3f/55/593da8d20b5c2810baf2a81c8ae78449b348f09c83111c51a3
Successfully built PyQt5-sip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nbconvert

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install keras_tuner

  Using cached keras_tuner-1.4.7-py3-none-any.whl.metadata (5.4 kB)
  Using cached kt_legacy-1.0.5-py3-none-any.whl.metadata (221 bytes)
Using cached keras_tuner-1.4.7-py3-none-any.whl (129 kB)
Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)
Note: you may need to restart the kernel to use updated packages.
